<link rel="stylesheet" href="../../styles/theme_style.css">
<!--link rel="stylesheet" href="../../styles/header_style.css"-->
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<table width="100%">
    <tr>
        <td id="image_td" width="15%" class="header_image_color_5"><div id="image_img" class="header_image_5"></div></td>
        <td class="header_text"> Event Detection - R Peaks (ECG) </td>
    </tr>
</table>

<div id="flex-container">
    <div id="diff_level" class="flex-item">
        <strong>Difficulty Level:</strong>   <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star checked"></span>
                                <span class="fa fa-star"></span>
    </div>
    <div id="tag" class="flex-item-tag">
        <span id="tag_list">
            <table id="tag_list_table">
                <tr>
                    <td class="shield_left">Tags</td>
                    <td class="shield_right" id="tags">detect&#9729;ecg&#9729;r-peaks</td> 
                </tr>
            </table>
        </span>
        <!-- [OR] Visit https://img.shields.io in order to create a tag badge-->
    </div>
</div>

One of the distinctive characteristics of the electrocardiographic (ECG) signals is their periodicity, something that is not common in physiological terms.

Due to this characteristic, the study of cardiac cycle variability (heart rate variability) defines an important segment of ECG analysis.

However, heart rate variability analysis is dependent of the detection of ECG R peaks, which is the main topic of the present **<span class="color5">Jupyter Notebook</span>**. This task can be achieved by applying the Pan-Tompkins algorithm, translated to **<span class="color1">Python</span>** paradigm by <a href="https://github.com/RajaS/ecgtk">Raja Selvaraj <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>

<hr>

<p class="steps">1 - Importation of the needed packages and definition of auxiliary functions</p>

In [1]:
# biosignalsnotebooks python package
import biosignalsnotebooks as bsnb

# Scientific packages
from numpy import linspace, diff, zeros_like, arange, array

<p class="steps">2 - Load of acquired ECG data</p>

In [6]:

# Data loading
data, header = bsnb.load("../data/ECG-ejer_andrea.h5", get_header=True)

print ("\033[1mHeader:\n\033[0m" + str(header) + "\n\033[1mData:\033[0m\n" + str(data))

Header:
{'channels': array([2]), 'comments': '', 'date': '2024-2-28', 'device': 'bitalino_rev', 'device connection': 'BTH0C:43:14:1C:2A:25', 'device name': '0C:43:14:1C:2A:25', 'digital IO': array([0, 0, 1, 1]), 'firmware version': 1282, 'resolution': array([ 4,  1,  1,  1,  1, 10]), 'sampling rate': 1000, 'sync interval': 2, 'time': '11:38:24.655', 'sensor': ['ECGBIT'], 'column labels': {2: 'channel_2'}}
Data:
{'CH2': array([498, 498, 498, ..., 507, 507, 507], dtype=uint32)}


In [7]:


ch = list(data.keys())[0]
print(f'Channel: {ch}')
sr = header["sampling rate"] # Sampling rate
print(f'Sampling rate: {sr}')
resolution = header["resolution"][-1] # Resolution
print(f'Resolution: {resolution}')

Channel: CH2
Sampling rate: 1000
Resolution: 10


<p class="steps">4 - Storage of sampling rate and acquired data inside variables</p>

In [8]:
# Acquired data
signal = data[ch]

<p class="steps">4.1 - A time-axis should be generated in order to a more intuitive interpretation of the final results</p>

In [9]:
time = bsnb.generate_time(signal)

<p class="steps">5 - Simplification of ECG signal (isolation of abrupt transitions)
<br>5.1 - Step 1 of Pan-Tompkins Algorithm - ECG Filtering (Bandpass between 5 and 15 Hz)</p>

In [10]:
# Step 1 of Pan-Tompkins Algorithm
filtered_signal = bsnb.detect._ecg_band_pass_filter(signal, sr)

In [11]:
bsnb.plot_ecg_pan_tompkins_steps(time, signal, filtered_signal, sr, titles=["Original Signal", "Post Filtering Signal"])

<p class="steps">5.2 - Step 2 of Pan-Tompkins Algorithm - ECG Differentiation</p>

In [12]:
# Step 2 of Pan-Tompkins Algorithm
differentiated_signal = diff(filtered_signal)

In [13]:
bsnb.plot_ecg_pan_tompkins_steps(time, filtered_signal, differentiated_signal, sr, titles=["Post Filtering Signal", "Post Differentiation Signal"])

<p class="steps">5.3 - Step 3 of Pan-Tompkins Algorithm - ECG Rectification</p>

In [14]:
# Step 3 of Pan-Tompkins Algorithm
squared_signal = differentiated_signal * differentiated_signal

In [15]:
bsnb.plot_ecg_pan_tompkins_steps(time, filtered_signal, squared_signal, sr, titles=["Post Differentiation Signal", "Post Rectification Signal"])

<p class="steps">5.4 - Step 4 of Pan-Tompkins Algorithm - ECG Integration ( <i>Moving window integration</i> )
<br>5.4.1 - Definition of the samples number inside the moving average window</p>

In [16]:
nbr_sampls_int_wind = int(0.080 * sr)

<p class="steps">5.4.2 - Initialisation of the variable that will contain the integrated signal samples</p>

In [17]:
integrated_signal = zeros_like(squared_signal)

<p class="steps">5.4.3 - Determination of a cumulative version of "squared_signal"</p>
In the cumulative version of the signal under analysis, his sample value $i$ will be sum of all values included between entry 0 and entry $i$ of the studied signal (in our case "squared_signals"). 

In [18]:
cumulative_sum = squared_signal.cumsum()

<p class="steps">5.4.4 - Estimation of the area/integral below the curve that defines the "squared_signal"</p>
Implicitly, with the current procedure, "squared_signal" is divided into multiple rectangles with fixed width (equal 1 sample) and <a href="https://en.wikipedia.org/wiki/Riemann_sum">height determined by the sample value under analysis <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>.

In [19]:
integrated_signal[nbr_sampls_int_wind:] = (cumulative_sum[nbr_sampls_int_wind:] - 
                                           cumulative_sum[:-nbr_sampls_int_wind]) / nbr_sampls_int_wind
integrated_signal[:nbr_sampls_int_wind] = cumulative_sum[:nbr_sampls_int_wind] / arange(1, nbr_sampls_int_wind + 1)

In [20]:
bsnb.plot_ecg_pan_tompkins_steps(time, squared_signal, integrated_signal, sr, titles=["Post Rectification Signal", "Post Integration Signal"])

<p class="steps">6 - Application of the ECG simplified version to a sequence of peak identification steps</p>
This task is achieved by using a threshold system, with this threshold being dynamically adjusted along the acquisition, like originally proposed by <a href="http://www.robots.ox.ac.uk/~gari/teaching/cdt/A3/readings/ECG/Pan+Tompkins.pdf">Pan and Tompkins <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a>

<p class="steps">6.1 - Initialisation of the R peak detection algorithm</p>

In [21]:
rr_buffer, signal_peak_1, noise_peak_1, threshold = bsnb.detect._buffer_ini(integrated_signal, sr)

<p class="steps">6.2 - Detection of <span class="color7">possible</span> and <span class="color4">probable</span> R peaks</p>
Each sample $i$ of our signal is a <strong><span class="color7">possible</span></strong> peak if his value is greater than the ones at sample $i-1$ and $i+1$. On the other hand, a <strong><span class="color4">probable</span></strong> peak is a <strong><span class="color7">possible</span></strong> peak that meets some criteria defined by Pan and Tompkins at their original <a href="http://www.robots.ox.ac.uk/~gari/teaching/cdt/A3/readings/ECG/Pan+Tompkins.pdf">article <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a> and synthesised inside <strong>_detect_peaks</strong> function of <span class="color1"><strong>biosignalsnotebooks</strong></span> Python package.

In [22]:
probable_peaks, possible_peaks= bsnb.detect._detects_peaks(integrated_signal, sr)

Entry "0" of the returned result contains the list of probable peaks, while entry "1" refers to the possible_peaks.

<p class="steps">6.3 - Identification of definitive R peaks</p>
Taken into consideration the list of previously detected <span class="color4"><strong>probable</strong></span> peaks, a set of additional criteria were defined by Pan and Tompkins, in order to exclude peaks from the list of <span class="color4"><strong>probable</strong></span> peaks.

In [23]:
definitive_peaks = bsnb.detect._checkup(probable_peaks, integrated_signal, sr, rr_buffer, signal_peak_1, noise_peak_1, threshold)

# Conversion to integer type.
definitive_peaks = array(list(map(int, definitive_peaks)))

<p class="steps">6.5 - Correcting step</p>
Due to the multiple pre-processing stages there is a small lag in the determined peak positions, which needs to be corrected !

In [24]:
map_integers = definitive_peaks - 40 * (sr / 1000)
definitive_peaks_reph = array(list(map(int, map_integers)))

<p class="steps">7 - Evolution of the detected peaks (from possible to definitive R peaks)</p>

In [25]:
bsnb.plot_ecg_pan_tompkins_peaks(time, signal, integrated_signal, sr, possible_peaks, probable_peaks, definitive_peaks)

<i>This procedure can be automatically done by <strong>detect_r_peaks</strong> function in <strong>detect</strong> module of <strong><span class="color2">opensignalstools</span></strong> package</i>

In [26]:
detected_peaks = bsnb.detect_r_peaks(signal, sr, time_units=True, plot_result=True)

As demonstrated, R peak detection algorithm contains a great number of stages, obviously there are alternative and simpler methodologies. However, to obtain more precise results, a bigger complexity becomes mandatory !

Now you can understand better the complexity behind the heart beat monitoring systems but you can also program it.

<strong><span class="color7">We hope that you have enjoyed this guide. </span><span class="color2">biosignalsnotebooks</span><span class="color4"> is an environment in continuous expansion, so don't stop your journey and learn more with the remaining <a href="../MainFiles/biosignalsnotebooks.ipynb">Notebooks <img src="../../images/icons/link.png" width="10px" height="10px" style="display:inline"></a></span></strong> ! 

<strong><span class="color6">Auxiliary Code Segment</span></strong>

In [27]:
from biosignalsnotebooks.__notebook_support__ import css_style_apply
css_style_apply()

.................... CSS Style Applied to Jupyter Notebook .........................


In [29]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>